In [1]:
from pathlib import Path
import os
from dotenv import load_dotenv
from tqdm import tqdm
tqdm.pandas()
%load_ext autoreload
%autoreload 2

In [2]:
# change dir to ml repo at ml/ root (if you started in notebook/)
os.chdir('../')
print(Path.cwd())
load_dotenv()

/home/opadmin/llama2d_opentable/lm_act_eval


True

In [3]:
# read dataset into datasets
from datasets import Dataset
import pandas as pd
dataset_name = "five-star-trajectories"
dataset_path = Path(f"lm_act_eval/.cache/{dataset_name}")
traj_df = pd.read_csv(dataset_path/"csv/data.csv")
traj_dataset = Dataset.from_pandas(traj_df)
print(traj_df.columns)
# traj_df[traj_df.session_id==traj_df.session_id.unique()[0]]
traj_df.head(2)

/home/opadmin/anaconda3/envs/eval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Index(['Unnamed: 0', '_id', 'tenant', 'project_id', 'created_at', 'updated_at',
       'inputs', 'history', 'ground_truth', 'linked_event', 'linked_evals',
       'linked_datasets', 'saved', 'type', 'metadata', 'QUERY', 'URL', 'DOM',
       'RULES', 'CURRENT_TIME', 'planPrompt',
       'previousActionsRepetitionPrompt', 'chat_completion_messages',
       'USER_CONTEXT', 'PREV_ACTIONS', 'prompt', 'CHAT_HISTORY'],
      dtype='object')


,Unnamed: 0,_id,tenant,project_id,created_at,updated_at,inputs,history,ground_truth,linked_event,...,DOM,RULES,CURRENT_TIME,planPrompt,previousActionsRepetitionPrompt,chat_completion_messages,USER_CONTEXT,PREV_ACTIONS,prompt,CHAT_HISTORY
0,0,65c95958a3d1e05a556776d4,org_5486oZLB8HZcT2Y1,648f6f2cd97aa222cef53da9,2024-02-11 23:33:44.479,2024-02-11 23:33:44.479,"{'QUERY': ""Book a reservation for the restaura...","[{'role': 'user', 'content': ""Book a reservati...",It appears that the browser content is not ava...,9c92b671-c113-4472-aeb9-d1d5944f1a9b,...,NaN,NaN,"Sunday, February 11, 2024 at 4:44:52 AM UTC","IMPORTANT: If the objective is complex, or you...",**PREVIOUS ACTIONS Repetition**:\n- Review the...,"[{'role': 'system', 'content': '**Goal**: Let\...",NaN,NaN,NaN,NaN
1,1,65c95958a3d1e05a556776d5,org_5486oZLB8HZcT2Y1,648f6f2cd97aa222cef53da9,2024-02-11 23:33:44.480,2024-02-11 23:33:44.480,"{'QUERY': ""Book a reservation for the restaura...","[{'role': 'user', 'content': ""Book a reservati...",COMMANDS:\nCLICK 29\nEXPLANATION: I am clickin...,535be7d9-b6ad-4014-8191-a293ee71372d,...,<l id=0/>\n<t id=1>Skip to main content/>\n<l ...,NaN,"Sunday, February 11, 2024 at 4:44:48 AM UTC","IMPORTANT: If the objective is complex, or you...",**PREVIOUS ACTIONS Repetition**:\n- Review the...,"[{'role': 'system', 'content': '**Goal**: Let\...",NaN,NaN,NaN,NaN


In [4]:
[
  print(obj[15]) for obj in [
    traj_df.outputs, 
    traj_df.ground_truth, 
    traj_df.PREV_ACTIONS
  ]
]

AttributeError: 'DataFrame' object has no attribute 'outputs'

In [48]:
from lm_act_eval.evaluation_harness.helper_functions.multion import (
  action_prefix,
  clean_extracted_text,
  extract_thought,
  extract_action,
  extract_explanation,
  ParseChatCompletion,
  extract_first
)
from lm_act_eval.evaluation_harness.utils.url import is_screenshot_url_accessible

In [ ]:
for i in [
  traj_df.PREV_ACTIONS,
  traj_df.previousActionsRepetitionPrompt,
  traj_df.chat_completion_messages,
  traj_df.ground_truth,
  traj_df.screenshot
  ]:
  if type(i[2])==str:
    print(i[2].replace('\n\n','\n'))
  else:
    print(i[2])
  print("")
  

In [5]:
from typing import Callable
from tqdm import tqdm

In [ ]:
# skip 
def criteria(eligibility_func: callable, cols: list, description: str=None):
  return pd.Series([eligibility_func(item) for item in tqdm(traj_df[cols], desc=description)])

def is_eligible():  
  # Apply the function to each URL in the column
  is_screenshot_accessible = criteria(is_screenshot_url_accessible, 'screenshot',"determine URL accessibility")
  return is_screenshot_accessible

eligible = is_eligible()
# Calculate the percentage of accessible URLs
percentage_accessible = (eligible.sum() / len(traj_df)) * 100
print(percentage_accessible)


determine URL accessibility: 100%|██████████| 3382/3382 [1:02:27<00:00,  1.11s/it]

4.376108811354229


In [6]:
from lm_act_eval.evaluation_harness.openai.vision.gptv import GPTV
from lm_act_eval.evaluation_harness.openai.vision.config import gptv_config
gptv_pipeline = GPTV(gptv_config)

Google Cloud not set up, skipping import of providers.gemini_utils.generate_from_gemini_completion
Google Cloud not set up, skipping import of vertexai.preview.generative_models.Image and llms.generate_from_gemini_completion


/home/opadmin/anaconda3/envs/eval/lib/python3.11/site-packages/deepeval/__init__.py:41: UserWarning: You are using deepeval version 0.20.90, however version 0.21.13 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


In [7]:
eligible_traj_df = pd.read_csv(dataset_path/"csv/data+gptv-eligible.csv")
eligible_traj_df = eligible_traj_df.set_index(eligible_traj_df.columns[0])
# or 
# eligible_traj_df = traj_df[eligible]
len(eligible_traj_df)
eligible_traj_df.head(2)

,session_id,idx_in_session,_id,event_id,parent_id,event_type,event_name,config,children_ids,inputs,...,CURRENT_TIME,planPrompt,previousActionsRepetitionPrompt,chat_completion_messages,ground_truth,screenshot,prompt,internal_metadata,chat_completion_messages_content,GPTV_generations
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
169,36c2f0ba-e215-4081-96b4-ecb04c10a517,0,65ed4fc8166e07fe3c29c476,6c8932f3-c31d-4683-aeb0-52a5bbe5a978,36c2f0ba-e215-4081-96b4-ecb04c10a517,model,Model Completion,"{'model': 'gpt-4-1106-preview', 'provider': 'u...",[],{'QUERY': 'What was the actual enrollment coun...,...,"Sunday, 10 March 2024 at 17:14:14 GMT+11","IMPORTANT: If the objective is complex, compri...",**PREVIOUS ACTIONS Repetition**:\n- Review the...,"[{'role': 'system', 'content': '**Goal**: Let\...",PLAN:\n1. Use the Google search function to fi...,https://multion-client-screenshots.s3.us-east-...,NaN,NaN,**Goal**: Let's play a game - You are an exper...,PLAN: The current user objective is to find th...
170,36c2f0ba-e215-4081-96b4-ecb04c10a517,1,65ed4fd1b2b5b0c1673c47d7,172ff25e-efdb-4002-b43a-0180f3f0eb19,36c2f0ba-e215-4081-96b4-ecb04c10a517,model,Model Completion,"{'model': 'gpt-4-1106-preview', 'provider': 'u...",[],{'QUERY': 'What was the actual enrollment coun...,...,"Sunday, 10 March 2024 at 17:14:35 GMT+11","IMPORTANT: If the objective is complex, compri...",**PREVIOUS ACTIONS Repetition**:\n- Review the...,"[{'role': 'system', 'content': '**Goal**: Let\...",COMMANDS:\nCLICK 26\nEXPLANATION: I am clickin...,https://multion-client-screenshots.s3.us-east-...,NaN,NaN,**Goal**: Let's play a game - You are an exper...,COMMANDS:\nCLICK 26\nEXPLANATION: I am clickin...


In [8]:
groundtruth, chat_completions, screenshots, user_inputs = eligible_traj_df.ground_truth, eligible_traj_df.chat_completion_messages, eligible_traj_df.screenshot, eligible_traj_df.inputs
eligible_idx = eligible_traj_df.index

In [13]:
# Using as agent
from lm_act_eval.evaluation_harness.helper_functions.multion import extract_explanation

eval_input_df = pd.DataFrame(index=eligible_traj_df.index) 
# Using as evaluator
eval_input_df['QUERY'] = user_inputs.apply(lambda s: ParseChatCompletion().parse_as_json(
  s, target_field=None).get('QUERY',''))
eval_input_df['GOAL'] = chat_completions.apply(lambda s: ParseChatCompletion().parse_as_completion_content(s))

In [14]:
# evaluator prompt
eval_prompt = "Start with 'SCORE:\n' and rate the screenshot image on 0-100 based on how close a user is to completing the task followed by 'EXPLANATION:\n' in a new line with an explanation. The user were given the following web navigation task:\n{GOAL}. With the intention to do the following task in the web:{QUERY}.\n"
eval_query = eval_input_df.apply(lambda r: eval_prompt.format(**r), axis=1)


In [17]:
input_df = pd.DataFrame(index=eligible_traj_df.index)
input_df['text'] = eval_query
input_df['images'] = screenshots

In [ ]:
import re
evals = input_df.iloc[:5].progress_apply(lambda r: gptv_pipeline.generate_completion(**r), axis=1)

In [46]:
result_df = evals.str.split('\n', n=1, expand=True)
result_df.columns = ['Score', 'Explanation']
result_df.head(2)

,Score,Explanation
Unnamed: 0,,
169,SCORE: 90,EXPLANATION:\nThe user is very close to comple...
170,SCORE: 95,EXPLANATION:\nThe user is very close to comple...


In [50]:
result_df['Score'] = result_df['Score'].apply(extract_first(term='SCORE'))
result_df['Explanation'] = result_df['Explanation'].apply(extract_first(term='EXPLANATION'))

TypeError: extract_first() missing 1 required positional argument: 'text'

In [ ]:
extract_score = lambda x: int(re.findall(r'SCORE:\s*(\d+)', x)[0]) if re.search(r'SCORE:\s*(\d+)', x) else None
result_df[0] = result_df[0].apply(extract_score)

In [ ]:
import warnings

completion_responses = []
for i in tqdm(eligible_idx, desc="Generating completions"):
  completion = gptv_pipeline.generate_completion(
    eval_query.loc[i],   #chat_completions_processed[i],
    screenshots.loc[i]
  )
  if len(completion) > 1:
    warnings.warn("Completion length more than 0 is unexpected")
  completion_responses.append(completion[0].message.content)
  

Generating completions:   0%|          | 0/148 [00:00<?, ?it/s]

Generating completions: 100%|██████████| 148/148 [29:00<00:00, 11.76s/it]


In [ ]:
eligible_traj_df.loc[:, 'GPTV_evaluations'] = completion_responses

## Save/Load Data

In [ ]:
# traj_df_new = traj_df.merge(eligible_traj_df, how='left')
# traj_df_new.to_csv(dataset_path/'csv/data+gptv_eval.csv')

In [ ]:
table_name = 'data+gptv_eval-eligible'
artifact_name = 'gptv_opentable-trajectory_eval'
eligible_traj_df.to_csv(dataset_path/f'csv/{table_name}.csv')

In [ ]:
eligible_traj_df = pd.read_csv(
  dataset_path/f'csv/{table_name}.csv', index_col=0)
eligible_traj_df.index.name = None
eligible_traj_df.head(2)

,session_id,idx_in_session,_id,event_id,parent_id,event_type,event_name,config,children_ids,inputs,...,planPrompt,previousActionsRepetitionPrompt,chat_completion_messages,ground_truth,screenshot,prompt,internal_metadata,GPTV response,chat_completion_messages_content,GPTV_evaluation
169,36c2f0ba-e215-4081-96b4-ecb04c10a517,0,65ed4fc8166e07fe3c29c476,6c8932f3-c31d-4683-aeb0-52a5bbe5a978,36c2f0ba-e215-4081-96b4-ecb04c10a517,model,Model Completion,"{'model': 'gpt-4-1106-preview', 'provider': 'u...",[],{'QUERY': 'What was the actual enrollment coun...,...,"IMPORTANT: If the objective is complex, compri...",**PREVIOUS ACTIONS Repetition**:\n- Review the...,"[{'role': 'system', 'content': '**Goal**: Let\...",PLAN:\n1. Use the Google search function to fi...,https://multion-client-screenshots.s3.us-east-...,NaN,NaN,It appears that you are looking for informatio...,**Goal**: Let's play a game - You are an exper...,SCORE: 60\nEXPLANATION:\nThe user appears to b...
170,36c2f0ba-e215-4081-96b4-ecb04c10a517,1,65ed4fd1b2b5b0c1673c47d7,172ff25e-efdb-4002-b43a-0180f3f0eb19,36c2f0ba-e215-4081-96b4-ecb04c10a517,model,Model Completion,"{'model': 'gpt-4-1106-preview', 'provider': 'u...",[],{'QUERY': 'What was the actual enrollment coun...,...,"IMPORTANT: If the objective is complex, compri...",**PREVIOUS ACTIONS Repetition**:\n- Review the...,"[{'role': 'system', 'content': '**Goal**: Let\...",COMMANDS:\nCLICK 26\nEXPLANATION: I am clickin...,https://multion-client-screenshots.s3.us-east-...,NaN,NaN,The image shows a webpage from ClinicalTrials....,**Goal**: Let's play a game - You are an exper...,SCORE: 95\nEXPLANATION:\nThe user is very clos...


Logging

In [ ]:
# log data
import wandb
from dotenv import load_dotenv
load_dotenv()
wandb.login(relogin=True)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /home/opadmin/.netrc


True

In [ ]:
opentable_artifact = wandb.Artifact("opentable_trajectories-gptv_eval", type="dataset")
# opentable_table = wandb.Table(dataframe=traj_df_new)
opentable_table_eligible = wandb.Table(dataframe=eligible_traj_df)
#
# opentable_artifact.add(opentable_table, "opentable")
opentable_artifact.add(
  opentable_table_eligible, artifact_name)
# opentable_artifact.add_file(str(dataset_path/'csv/data+gptv.csv'))
opentable_artifact.add_file(str(dataset_path/f'csv/{table_name}.csv'))


In [ ]:
# Start a W&B run to log data
run = wandb.init(project="trajectory_eval", reinit=True)
# Log the table to visualize with a run...
run.log({"opentable_gptv": opentable_table_eligible})

# and Log as an Artifact to increase the available row limit!
run.log_artifact(opentable_artifact)

wandb: Currently logged in as: mmym (multion-agi). Use `wandb login --relogin` to force relogin


wandb: WARNING Serializing object of type str that is 531752 bytes
wandb: WARNING Serializing object of type str that is 517908 bytes
wandb: WARNING Serializing object of type str that is 541284 bytes
wandb: WARNING Serializing object of type str that is 525810 bytes
wandb: WARNING Serializing object of type str that is 116864 bytes
wandb: WARNING Serializing object of type str that is 121274 bytes
wandb: WARNING Serializing object of type str that is 120688 bytes
wandb: WARNING Serializing object of type str that is 124364 bytes
wandb: WARNING Serializing object of type str that is 127354 bytes
wandb: WARNING Serializing object of type str that is 129308 bytes
wandb: WARNING Serializing object of type str that is 129582 bytes
wandb: WARNING Serializing object of type str that is 135310 bytes
wandb: WARNING Serializing object of type str that is 137856 bytes
wandb: WARNING Serializing object of type str that is 137644 bytes
wandb: WARNING Serializing object of type str that is 136800 b

<Artifact opentable_trajectories-gptv_eval>